In [ ]:
# Unzip the file
import zipfile
import os

zip_path = '/content/drive/MyDrive/ChinTranslator.zip'
extract_path = '/content/drive/MyDrive/'

print("📦 Unzipping ChinTranslator.zip...")
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("✅ Done!")

# Verify it worked
folder_path = '/content/drive/MyDrive/ChinTranslator'
print(f"\n📂 Folder exists: {os.path.exists(folder_path)}")

if os.path.exists(folder_path):
    print("\nContents:")
    for item in os.listdir(folder_path):
        print(f"  • {item}")

📦 Unzipping ChinTranslator.zip...
✅ Done!

📂 Folder exists: True

Contents:
  • training_data.json
  • requirements.md
  • fine-tuning.py
  • .DS_Store
  • train-validation.py
  • SetupWhisper.py
  • Audio
  • .venv
  • Text


In [ ]:
# Go to your ChinTranslator folder
%cd /content/drive/MyDrive/ChinTranslator

# Verify you're in the right place
!pwd
!ls -la

/content/drive/MyDrive/ChinTranslator
/content/drive/MyDrive/ChinTranslator
total 114
drwx------ 2 root root  4096 Oct 31 00:46 Audio
-rw------- 1 root root  6148 Oct 31 00:46 .DS_Store
-rw------- 1 root root  6604 Oct 31 00:46 fine-tuning.py
-rw------- 1 root root  1985 Oct 31 00:46 requirements.md
-rw------- 1 root root  1696 Oct 31 00:46 SetupWhisper.py
drwx------ 2 root root  4096 Oct 31 00:46 Text
-rw------- 1 root root 84970 Oct 31 00:46 training_data.json
-rw------- 1 root root  1237 Oct 31 00:46 train-validation.py
drwx------ 6 root root  4096 Oct 31 00:46 .venv


In [ ]:
import os
import json
from pathlib import Path

# Set up paths
base_dir = Path(".")
audio_dir = base_dir / "Audio"
text_dir = base_dir / "Text"
output_file = base_dir / "training_data.json"

# Function to read text file
def read_text_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        return f.read().strip()

# Create dataset
dataset = []

# Get all text files
text_files = sorted(text_dir.glob("*.txt"))

print(f"Found {len(text_files)} text files")
print("\nProcessing files...\n")

for text_file in text_files:
    # Get corresponding audio file
    audio_file = audio_dir / (text_file.stem + ".mp3")

    # Check if audio file exists
    if not audio_file.exists():
        print(f"⚠️  WARNING: No audio file found for {text_file.name}")
        continue

    # Read the text
    text = read_text_file(text_file)

    # Create data entry
    entry = {
        "audio": str(audio_file),
        "text": text,
        "chapter": text_file.stem
    }

    dataset.append(entry)
    print(f"✓ Paired: {text_file.name} ↔ {audio_file.name}")
    print(f"  Text length: {len(text)} characters")
    print(f"  Text preview: {text[:100]}...")
    print()

# Save dataset
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(dataset, f, ensure_ascii=False, indent=2)

print(f"\n{'='*50}")
print(f"✅ Dataset created successfully!")
print(f"{'='*50}")
print(f"Total pairs: {len(dataset)}")
print(f"Saved to: {output_file}")
print(f"\nNext steps:")
print(f"1. Review the output file to verify everything looks correct")
print(f"2. Check audio file durations match text length expectations")
print(f"3. Ready to start Whisper fine-tuning!")

Found 16 text files

Processing files...

✓ Paired: mark_01.txt ↔ mark_01.mp3
  Text length: 5567 characters
  Text preview: Pathian Fapa Jesuh Khrih kong Thawngṭha cu hitihin aa thawkning a si.
Profet Isaiah nih, "Pathian ni...

✓ Paired: mark_02.txt ↔ mark_02.mp3
  Text length: 4251 characters
  Text preview: Cu hnu ni tlawmpal ah khan Jesuh cu Kapernaum khua ah khan a ra ṭhan i a inn ah a um ti thawng kha a...

✓ Paired: mark_03.txt ↔ mark_03.mp3
  Text length: 4176 characters
  Text preview: Jesuh cu pumhnak inn ah khan a va kal ṭhan i cuka ah cun a kut a zengmipa pakhat a rak um. Jesuh cu ...

✓ Paired: mark_04.txt ↔ mark_04.mp3
  Text length: 5368 characters
  Text preview: Jesuh nih Galilee Rili kamah khan a cawnpiak ṭhan hna. A velchum i aa pummi cu an tam tuk caah lawng...

✓ Paired: mark_05.txt ↔ mark_05.mp3
  Text length: 5166 characters
  Text preview: Cuticun Galilee Rili ral, Gerasa mi hna ram cu an va phan. Jesuh cu lawng chungin a chuah cangka kha...

✓ Paired: mark_06.

In [ ]:
import json
import random
from pathlib import Path

# Load the training data
with open('./training_data.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

print(f"Total chapters: {len(data)}")

# Set random seed for reproducibility
random.seed(42)

# Shuffle the data
random.shuffle(data)

# Create 80/20 split (13 train, 3 validation)
split_idx = int(len(data) * 0.8)
train_data = data[:split_idx]
val_data = data[split_idx:]

print(f"\nTrain set: {len(train_data)} chapters")
print(f"Validation set: {len(val_data)} chapters")

# Show which chapters are in each set
train_chapters = [item['chapter'] for item in train_data]
val_chapters = [item['chapter'] for item in val_data]

print(f"\nTrain chapters: {', '.join(sorted(train_chapters))}")
print(f"Validation chapters: {', '.join(sorted(val_chapters))}")

# Save the splits
output_dir = Path('.')

with open(output_dir / 'train_data.json', 'w', encoding='utf-8') as f:
    json.dump(train_data, f, ensure_ascii=False, indent=2)

with open(output_dir / 'val_data.json', 'w', encoding='utf-8') as f:
    json.dump(val_data, f, ensure_ascii=False, indent=2)

print(f"\n✅ Created:")
print(f"  - {output_dir / 'train_data.json'}")
print(f"  - {output_dir / 'val_data.json'}")

Total chapters: 16

Train set: 12 chapters
Validation set: 4 chapters

Train chapters: mark_02, mark_03, mark_06, mark_07, mark_08, mark_09, mark_10, mark_11, mark_13, mark_14, mark_15, mark_16
Validation chapters: mark_01, mark_04, mark_05, mark_12

✅ Created:
  - train_data.json
  - val_data.json


In [ ]:
# Install all required packages
!pip install -q transformers datasets accelerate evaluate jiwer librosa soundfile tensorboard

In [ ]:
!pip install -q torchcodec

In [ ]:
!pip install -q librosa soundfile torchaudio

In [ ]:
"""
Whisper Fine-tuning Script for Hakha Chin
Run this in Google Colab or on a machine with GPU
"""

import torch
import json
from pathlib import Path
from datasets import Dataset, Audio
from transformers import (
    WhisperFeatureExtractor,
    WhisperTokenizer,
    WhisperProcessor,
    WhisperForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    TrainerCallback
)
from dataclasses import dataclass
from typing import Any, Dict, List, Union
import evaluate

# Configuration
MODEL_NAME = "openai/whisper-small"  # Start with small model
LANGUAGE = None  # Hakha Chin ISO code
TASK = "transcribe"
OUTPUT_DIR = "./whisper-hakha-chin"

print("="*50)
print("Whisper Fine-tuning for Hakha Chin")
print("="*50)

# Check for GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"\nUsing device: {device}")
if device == "cpu":
    print("⚠️  WARNING: No GPU detected. Training will be very slow!")
    print("   Consider using Google Colab with GPU enabled.")

# Load the data
print("\n📂 Loading data...")
with open('./train_data.json', 'r', encoding='utf-8') as f:
    train_data = json.load(f)
with open('./val_data.json', 'r', encoding='utf-8') as f:
    val_data = json.load(f)

print(f"Train samples: {len(train_data)}")
print(f"Validation samples: {len(val_data)}")

# Convert to HuggingFace Dataset format
def prepare_dataset(data_list):
    return Dataset.from_dict({
        "audio": [item["audio"] for item in data_list],
        "text": [item["text"] for item in data_list]
    })

train_dataset = prepare_dataset(train_data)
val_dataset = prepare_dataset(val_data)

# Cast audio column to Audio feature
train_dataset = train_dataset.cast_column("audio", Audio(sampling_rate=16000))
val_dataset = val_dataset.cast_column("audio", Audio(sampling_rate=16000))

print("✅ Datasets prepared")

# Load Whisper model components
print("\n🤖 Loading Whisper model...")
feature_extractor = WhisperFeatureExtractor.from_pretrained(MODEL_NAME)
tokenizer = WhisperTokenizer.from_pretrained(MODEL_NAME, language=LANGUAGE, task=TASK)
processor = WhisperProcessor.from_pretrained(MODEL_NAME, language=LANGUAGE, task=TASK)
model = WhisperForConditionalGeneration.from_pretrained(MODEL_NAME)
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

# Enable gradient checkpointing to save memory
model.config.use_cache = False
model.gradient_checkpointing_enable()

print("✅ Model loaded")

# Prepare data for training
def prepare_data(batch):
    # Load audio
    audio = batch["audio"]

    # Compute input features
    batch["input_features"] = feature_extractor(
        audio["array"],
        sampling_rate=audio["sampling_rate"]
    ).input_features[0]

    # Encode target text
    batch["labels"] = tokenizer(batch["text"]).input_ids

    return batch

print("\n🔄 Processing datasets...")
train_dataset = train_dataset.map(
    prepare_data,
    remove_columns=train_dataset.column_names
)
val_dataset = val_dataset.map(
    prepare_data,
    remove_columns=val_dataset.column_names
)
print("✅ Datasets processed")

# Data collator
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # Split into input features and labels
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        # Pad input features
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # Pad labels
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # Replace padding with -100 (ignore in loss)
        labels = labels_batch["input_ids"].masked_fill(
            labels_batch.attention_mask.ne(1), -100
        )

        # Remove BOS token if present
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels
        return batch

data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

# Evaluation metric
metric = evaluate.load("wer")

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # Replace -100 with pad token
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # Decode predictions and labels
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    # Compute WER
    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

# Training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=4,  # Adjust based on GPU memory
    gradient_accumulation_steps=2,  # Effective batch size = 8
    learning_rate=1e-5,
    warmup_steps=50,
    num_train_epochs=10,  # Adjust as needed
    gradient_checkpointing=True,
    fp16=True if device == "cuda" else False,  # Mixed precision training
    evaluation_strategy="steps",
    per_device_eval_batch_size=2,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=100,
    eval_steps=100,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
)

# Custom callback for progress
class ProgressCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs:
            print(f"Step {state.global_step}: {logs}")

# Initialize trainer
trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
    callbacks=[ProgressCallback()],
)

print("\n🚀 Starting training...")
print("="*50)

# Train!
trainer.train()

print("\n✅ Training complete!")
print(f"📁 Model saved to: {OUTPUT_DIR}")

# Save final model
trainer.save_model(OUTPUT_DIR)
processor.save_pretrained(OUTPUT_DIR)

print("\n🎉 All done! Your fine-tuned Hakha Chin Whisper model is ready!")
print(f"\nTo use it:")
print(f"  model = WhisperForConditionalGeneration.from_pretrained('{OUTPUT_DIR}')")
print(f"  processor = WhisperProcessor.from_pretrained('{OUTPUT_DIR}')")

Whisper Fine-tuning for Hakha Chin

Using device: cuda

📂 Loading data...
Train samples: 12
Validation samples: 4
✅ Datasets prepared

🤖 Loading Whisper model...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


✅ Model loaded

🔄 Processing datasets...


Map:   0%|          | 0/12 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2077 > 1024). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/4 [00:00<?, ? examples/s]

✅ Datasets processed


TypeError: Seq2SeqTrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'

In [ ]:
# Navigate back to your project folder
%cd /content/drive/MyDrive/ChinTranslator

# Verify you're in the right place
!ls -la

# Now run training
!python fine-tuning.py

/content/drive/MyDrive/ChinTranslator
total 201
drwx------ 2 root root  4096 Oct 31 00:46 Audio
-rw------- 1 root root  6148 Oct 31 00:46 .DS_Store
-rw------- 1 root root  6604 Oct 31 01:22 fine-tuning.py
drwx------ 2 root root  4096 Oct 31 01:05 .ipynb_checkpoints
-rw------- 1 root root  1985 Oct 31 00:46 requirements.md
-rw------- 1 root root  1696 Oct 31 01:07 SetupWhisper.py
drwx------ 2 root root  4096 Oct 31 00:46 Text
-rw------- 1 root root 61554 Oct 31 01:08 train_data.json
-rw------- 1 root root 84970 Oct 31 01:08 training_data.json
-rw------- 1 root root  1237 Oct 31 01:06 train-validation.py
-rw------- 1 root root 23418 Oct 31 01:08 val_data.json
drwx------ 6 root root  4096 Oct 31 00:46 .venv
2025-10-31 01:22:48.703886: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761873768.736143   14312 cuda_dnn.cc:8579] Unable to register c

In [ ]:
# Uninstall and reinstall with specific versions that work together
!pip uninstall -y datasets
!pip install datasets==2.14.0

Found existing installation: datasets 4.0.0
Uninstalling datasets-4.0.0:
  Successfully uninstalled datasets-4.0.0
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.2/492.2 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 15.0 MB/s eta 0:00:00
  Attempting uninstall: dill
    Found existing installation: dill 0.3.8
    Uninstalling dill-0.3.8:
      Successfully uninstalled dill-0.3.8
  Attempting uninstall: multiprocess
    Found existing installation: multiprocess 0.70.16
    Uninstalling multiprocess-0.70.16:
      Successfully uninstalled multiprocess-0.70.16


In [ ]:
"""
Whisper Fine-tuning Script for Hakha Chin
Run this in Google Colab or on a machine with GPU
"""

import torch
import json
from pathlib import Path
from datasets import Dataset, Audio
from transformers import (
    WhisperFeatureExtractor,
    WhisperTokenizer,
    WhisperProcessor,
    WhisperForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    TrainerCallback
)
from dataclasses import dataclass
from typing import Any, Dict, List, Union
import evaluate

# Configuration
MODEL_NAME = "openai/whisper-small"  # Start with small model
LANGUAGE = None  # Don't specify language - let model learn Hakha Chin
TASK = "transcribe"
OUTPUT_DIR = "./whisper-hakha-chin"

print("="*50)
print("Whisper Fine-tuning for Hakha Chin")
print("="*50)

# Check for GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"\nUsing device: {device}")
if device == "cpu":
    print("⚠️  WARNING: No GPU detected. Training will be very slow!")
    print("   Consider using Google Colab with GPU enabled.")

# Load the data
print("\n📂 Loading data...")
with open('./train_data.json', 'r', encoding='utf-8') as f:
    train_data = json.load(f)
with open('./val_data.json', 'r', encoding='utf-8') as f:
    val_data = json.load(f)

print(f"Train samples: {len(train_data)}")
print(f"Validation samples: {len(val_data)}")

# Convert to HuggingFace Dataset format
def prepare_dataset(data_list):
    return Dataset.from_dict({
        "audio": [item["audio"] for item in data_list],
        "text": [item["text"] for item in data_list]
    })

train_dataset = prepare_dataset(train_data)
val_dataset = prepare_dataset(val_data)

# Cast audio column to Audio feature
train_dataset = train_dataset.cast_column("audio", Audio(sampling_rate=16000))
val_dataset = val_dataset.cast_column("audio", Audio(sampling_rate=16000))

print("✅ Datasets prepared")

# Load Whisper model components
print("\n🤖 Loading Whisper model...")
feature_extractor = WhisperFeatureExtractor.from_pretrained(MODEL_NAME)
tokenizer = WhisperTokenizer.from_pretrained(MODEL_NAME, task=TASK)
processor = WhisperProcessor.from_pretrained(MODEL_NAME, task=TASK)
model = WhisperForConditionalGeneration.from_pretrained(MODEL_NAME)
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

# Enable gradient checkpointing to save memory
model.config.use_cache = False
model.gradient_checkpointing_enable()

print("✅ Model loaded")

# Prepare data for training
def prepare_data(batch):
    # Load audio
    audio = batch["audio"]

    # Compute input features
    batch["input_features"] = feature_extractor(
        audio["array"],
        sampling_rate=audio["sampling_rate"]
    ).input_features[0]

    # Encode target text
    batch["labels"] = tokenizer(batch["text"]).input_ids

    return batch

print("\n🔄 Processing datasets...")
train_dataset = train_dataset.map(
    prepare_data,
    remove_columns=train_dataset.column_names
)
val_dataset = val_dataset.map(
    prepare_data,
    remove_columns=val_dataset.column_names
)
print("✅ Datasets processed")

# Data collator
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # Split into input features and labels
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        # Pad input features
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # Pad labels
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # Replace padding with -100 (ignore in loss)
        labels = labels_batch["input_ids"].masked_fill(
            labels_batch.attention_mask.ne(1), -100
        )

        # Remove BOS token if present
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels
        return batch

data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

# Evaluation metric
metric = evaluate.load("wer")

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # Replace -100 with pad token
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # Decode predictions and labels
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    # Compute WER
    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

# Training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=4,  # Adjust based on GPU memory
    gradient_accumulation_steps=2,  # Effective batch size = 8
    learning_rate=1e-5,
    warmup_steps=50,
    num_train_epochs=10,  # Adjust as needed
    gradient_checkpointing=True,
    fp16=True if device == "cuda" else False,  # Mixed precision training
    eval_strategy="steps",  # Changed from evaluation_strategy
    per_device_eval_batch_size=2,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=100,
    eval_steps=100,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
)

# Custom callback for progress
class ProgressCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs:
            print(f"Step {state.global_step}: {logs}")

# Initialize trainer
trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
    callbacks=[ProgressCallback()],
)

print("\n🚀 Starting training...")
print("="*50)

# Train!
trainer.train()

print("\n✅ Training complete!")
print(f"📁 Model saved to: {OUTPUT_DIR}")

# Save final model
trainer.save_model(OUTPUT_DIR)
processor.save_pretrained(OUTPUT_DIR)

print("\n🎉 All done! Your fine-tuned Hakha Chin Whisper model is ready!")
print(f"\nTo use it:")
print(f"  model = WhisperForConditionalGeneration.from_pretrained('{OUTPUT_DIR}')")
print(f"  processor = WhisperProcessor.from_pretrained('{OUTPUT_DIR}')")

Whisper Fine-tuning for Hakha Chin

Using device: cuda

📂 Loading data...
Train samples: 12
Validation samples: 4
✅ Datasets prepared

🤖 Loading Whisper model...
✅ Model loaded

🔄 Processing datasets...


Map:   0%|          | 0/12 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2077 > 1024). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/4 [00:00<?, ? examples/s]

✅ Datasets processed


/tmp/ipython-input-436597271.py:188: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(



🚀 Starting training...


You're using a WhisperTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


ValueError: Labels' sequence length 3031 cannot exceed the maximum allowed length of 448 tokens.

In [ ]:
!pip install -q openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 18.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
"""
Use Whisper to get timestamps and align with Hakha Chin text
This creates properly chunked training data
"""

import whisper
import json
from pathlib import Path

print("="*50)
print("Creating Aligned Training Data")
print("="*50)

# Load pre-trained Whisper model (tiny for speed)
print("\n📥 Loading Whisper model...")
model = whisper.load_model("tiny")
print("✅ Model loaded")

# Load your original data
print("\n📂 Loading original data...")
with open('training_data.json', 'r', encoding='utf-8') as f:
    original_data = json.load(f)

print(f"Found {len(original_data)} chapters")

# Process each chapter
aligned_segments = []

for idx, chapter in enumerate(original_data):  # Start with first 3 chapters as test
    print(f"\n🔄 Processing {chapter['chapter']}...")

    audio_path = chapter['audio']
    full_text = chapter['text']

    # Transcribe with timestamps
    result = model.transcribe(
        audio_path,
        language=None,  # Let it detect
        task="transcribe",
        verbose=False
    )

    print(f"  Found {len(result['segments'])} segments")

    # Split text into sentences (rough split by periods)
    sentences = [s.strip() for s in full_text.split('.') if s.strip()]

    # Match Whisper segments with sentences
    # Simple approach: distribute sentences across segments
    if len(sentences) > 0:
        sentences_per_segment = max(1, len(sentences) // len(result['segments']))

        sentence_idx = 0
        for seg in result['segments']:
            # Take a few sentences for this segment
            end_idx = min(sentence_idx + sentences_per_segment, len(sentences))
            segment_text = '. '.join(sentences[sentence_idx:end_idx])

            if segment_text:  # Only add if we have text
                aligned_segments.append({
                    'audio': audio_path,
                    'start': seg['start'],
                    'end': seg['end'],
                    'text': segment_text,
                    'chapter': chapter['chapter']
                })

            sentence_idx = end_idx

            if sentence_idx >= len(sentences):
                break

    print(f"  Created {len(aligned_segments) - len([s for s in aligned_segments if s['chapter'] != chapter['chapter']])} aligned segments")

print(f"\n✅ Total aligned segments: {len(aligned_segments)}")

# Show sample
print("\n📋 Sample aligned segment:")
if aligned_segments:
    sample = aligned_segments[0]
    print(f"  Chapter: {sample['chapter']}")
    print(f"  Time: {sample['start']:.2f}s - {sample['end']:.2f}s")
    print(f"  Text: {sample['text'][:100]}...")

# Save aligned data
output_file = 'aligned_training_data.json'
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(aligned_segments, f, ensure_ascii=False, indent=2)

print(f"\n💾 Saved to: {output_file}")
print("\nNext steps:")
print("1. Review the aligned data")
print("2. If it looks good, process all 16 chapters")
print("3. Split into train/val")
print("4. Start fine-tuning!")

Creating Aligned Training Data

📥 Loading Whisper model...
✅ Model loaded

📂 Loading original data...
Found 16 chapters

🔄 Processing mark_01...
Detected language: Indonesian


 93%|█████████▎| 37722/40722 [00:55<00:04, 674.98frames/s]


  Found 130 segments
  Created 54 aligned segments

🔄 Processing mark_02...
Detected language: Thai


  4%|▍         | 1221/31221 [01:10<28:40, 17.44frames/s]


  Found 3 segments
  Created 3 aligned segments

🔄 Processing mark_03...
Detected language: Myanmar


...

  Found 265 segments
  Created 101 aligned segments

🔄 Processing mark_15...
Detected language: Myanmar


100%|██████████| 46769/46769 [00:17<00:00, 2740.47frames/s]


  Found 91 segments
  Created 60 aligned segments

🔄 Processing mark_16...
Detected language: Thai


 16%|█▋        | 3504/21504 [00:49<04:14, 70.84frames/s]

  Found 9 segments
  Created 9 aligned segments

✅ Total aligned segments: 676

📋 Sample aligned segment:
  Chapter: mark_01
  Time: 0.00s - 4.00s
  Text: Pathian Fapa Jesuh Khrih kong Thawngṭha cu hitihin aa thawkning a si...

💾 Saved to: aligned_training_data.json

Next steps:
1. Review the aligned data
2. If it looks good, process all 16 chapters
3. Split into train/val
4. Start fine-tuning!


In [ ]:
import json
import random

# Load aligned data
with open('aligned_training_data.json', 'r') as f:
    data = json.load(f)

# Shuffle
random.seed(42)
random.shuffle(data)

# 80/20 split
split_idx = int(len(data) * 0.8)
train_data = data[:split_idx]
val_data = data[split_idx:]

# Save
with open('aligned_train_data.json', 'w') as f:
    json.dump(train_data, f, ensure_ascii=False, indent=2)

with open('aligned_val_data.json', 'w') as f:
    json.dump(val_data, f, ensure_ascii=False, indent=2)

print(f"Train: {len(train_data)} segments")
print(f"Val: {len(val_data)} segments")
print("\n✅ Ready for training!")

Train: 540 segments
Val: 136 segments

✅ Ready for training!


In [ ]:
"""
Run this in your Colab notebook to list all project files
Copy the output and paste it back to me
"""

import os
import json
from pathlib import Path

print("="*60)
print("COLAB PROJECT FILES")
print("="*60)

# Get current directory
current_dir = os.getcwd()
print(f"\nCurrent Directory: {current_dir}\n")

# Function to get file size in human-readable format
def get_size(size_bytes):
    for unit in ['B', 'KB', 'MB', 'GB']:
        if size_bytes < 1024.0:
            return f"{size_bytes:.1f} {unit}"
        size_bytes /= 1024.0
    return f"{size_bytes:.1f} TB"

# List all files recursively
print("FILE STRUCTURE:")
print("-" * 60)

all_files = []
for root, dirs, files in os.walk(current_dir):
    # Skip hidden directories and __pycache__
    dirs[:] = [d for d in dirs if not d.startswith('.') and d != '__pycache__']

    level = root.replace(current_dir, '').count(os.sep)
    indent = '  ' * level
    folder_name = os.path.basename(root) or 'ROOT'
    print(f'{indent}📁 {folder_name}/')

    subindent = '  ' * (level + 1)
    for file in sorted(files):
        if not file.startswith('.'):
            filepath = os.path.join(root, file)
            size = os.path.getsize(filepath)
            print(f'{subindent}📄 {file} ({get_size(size)})')
            all_files.append({
                'path': filepath.replace(current_dir + '/', ''),
                'size': size,
                'size_readable': get_size(size)
            })

print("\n" + "="*60)
print(f"TOTAL FILES: {len(all_files)}")
print("="*60)

# Check for key training files
print("\n🔍 CHECKING KEY FILES:")
print("-" * 60)

key_files = [
    'aligned_training_data.json',
    'aligned_train_data.json',
    'aligned_val_data.json',
    'train_data.json',
    'val_data.json'
]

for key_file in key_files:
    if os.path.exists(key_file):
        size = os.path.getsize(key_file)
        print(f"✅ {key_file} ({get_size(size)})")

        # If it's a JSON file, show how many items
        try:
            with open(key_file, 'r') as f:
                data = json.load(f)
                if isinstance(data, list):
                    print(f"   └─ Contains {len(data)} items")
                    if len(data) > 0:
                        print(f"   └─ Sample keys: {list(data[0].keys())}")
        except:
            pass
    else:
        print(f"❌ {key_file} (NOT FOUND)")

print("\n" + "="*60)
print("📋 COPY ALL OF THE ABOVE OUTPUT AND SEND IT BACK")
print("="*60)

COLAB PROJECT FILES

Current Directory: /content/drive/MyDrive/ChinTranslator

FILE STRUCTURE:
------------------------------------------------------------
📁 ChinTranslator/
  📄 SetupWhisper.py (1.7 KB)
  📄 aligned_train_data.json (130.9 KB)
  📄 aligned_training_data.json (160.5 KB)
  📄 aligned_val_data.json (29.7 KB)
  📄 fine-tuning.py (6.5 KB)
  📄 requirements.md (1.9 KB)
  📄 split-aligned-data.py (618.0 B)
  📄 train-validation.py (1.2 KB)
  📄 train_data.json (60.1 KB)
  📄 training_data.json (83.0 KB)
  📄 val_data.json (22.9 KB)
  📄 whisper_alignment.py (3.0 KB)
  📁 Audio/
    📄 mark_01.mp3 (3.1 MB)
    📄 mark_02.mp3 (2.4 MB)
    📄 mark_03.mp3 (2.4 MB)
    📄 mark_04.mp3 (3.1 MB)
    📄 mark_05.mp3 (3.0 MB)
    📄 mark_06.mp3 (4.6 MB)
    📄 mark_07.mp3 (3.1 MB)
    📄 mark_08.mp3 (2.9 MB)
    📄 mark_09.mp3 (3.9 MB)
    📄 mark_10.mp3 (4.4 MB)
    📄 mark_11.mp3 (2.7 MB)
    📄 mark_12.mp3 (4.0 MB)
    📄 mark_13.mp3 (2.7 MB)
    📄 mark_14.mp3 (5.7 MB)
    📄 mark_15.mp3 (3.6 MB)
    📄 mark_16

In [ ]:
!python fine-tuning-aligned.py

2025-10-31 03:39:24.103605: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761881964.123305   49075 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761881964.129296   49075 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761881964.144440   49075 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761881964.144467   49075 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761881964.144471   49075 computation_placer.cc:177] computation placer alr

In [ ]:
!python continue_training.py

2025-10-31 04:26:35.868005: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761884795.889034   61065 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761884795.895356   61065 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761884795.914161   61065 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761884795.914189   61065 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761884795.914193   61065 computation_placer.cc:177] computation placer alr

In [ ]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import librosa
import torch

# Load model
processor = WhisperProcessor.from_pretrained("./whisper-hakha-chin-v2")
model = WhisperForConditionalGeneration.from_pretrained("./whisper-hakha-chin-v2")
model.to("cuda")

print("Model config:")
print(f"  forced_decoder_ids: {model.config.forced_decoder_ids}")
print(f"  suppress_tokens: {model.config.suppress_tokens}")

# Test on ACTUAL training file
audio, _ = librosa.load("Audio/mark_01.mp3", sr=16000, duration=10)  # First 10 seconds
input_features = processor(audio, sampling_rate=16000, return_tensors="pt").input_features
input_features = input_features.to("cuda")

# Generate
predicted_ids = model.generate(
    input_features,
    max_length=200,
    repetition_penalty=1.5,
    no_repeat_ngram_size=3
)
result = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]

print(f"\nTest result: {result}")

Model config:
  forced_decoder_ids: None
  suppress_tokens: None


Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
Transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English. This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`. See https://github.com/huggingface/transformers/pull/28687 for more details.
`generation_config` default values have been modified to match model-specific defaults: {'suppress_tokens': [], 'begin_suppress_tokens': [220, 50257]}. If this is not desired, please set these values explicitly.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
A custom logits processor of type <class 'transform


Test result: Mirimah ṭha hlak chung ah khan an ra


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/ChinTranslator

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/ChinTranslator


In [ ]:
!python gradio_interface.py

2025-10-31 09:01:19.183504: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761901279.205407   20136 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761901279.211921   20136 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761901279.228225   20136 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761901279.228254   20136 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761901279.228257   20136 computation_placer.cc:177] computation placer alr

In [ ]:
!pip install deep-translator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.7 MB/s eta 0:00:00
